# P5 - Convolutioneel neuraal netwerk

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential


## Data preparation

In [4]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print("x_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("x_test shape", X_test.shape)
print("y_test shape", y_test.shape)

x_train shape (50000, 32, 32, 3)
y_train shape (50000, 1)
x_test shape (10000, 32, 32, 3)
y_test shape (10000, 1)


Voor one hot encoding uit over de target values, en laten we ook de dataset normaliseren

In [5]:
# One hot encodign over de targets heen
Y_train = utils.to_categorical(y_train, 10)
Y_test = utils.to_categorical(y_test, 10)

# normaliseren van de data
X_train = utils.normalize(X_train)
X_test = utils.normalize(X_test)

## Modelling

In [ ]:
cnn = Sequential()

cnn.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32,32,1)))
cnn.add()